In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)

LR_tune,N_value = np.meshgrid(lr_tune,n_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)

lrn_tune = np.hstack((LR_tune,N_value))

In [3]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y
    

In [4]:
loss_thresh = 0.005

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [5]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [6]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True
        
        self.n = torch.tensor(n_val)
    
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [7]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [8]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [9]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [10]:
for tune_reps in range(25):
    max_reps = 10
    max_iter = 100
    label = "1D_SODE_Stan_tune"+str(tune_reps)

    N_f = 1000

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    alpha_full = []
    elapsed_time= np.zeros((max_reps,1))

    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))
    n_val = lrn_tune[tune_reps,1]

    for reps in range(max_reps):

        train_loss = []
        test_mse_loss = []
        test_re_loss =[]
        alpha_val = []

        'Generate Training data'
        torch.manual_seed(reps*36)
         #Total number of collocation points 
        
        

        layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
        PINN = Sequentialmodel(layers,n_val)
        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())

        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrn_tune[tune_reps,0], 
                                  max_iter = 10, 
                                  max_eval = 15, 
                                  tolerance_grad = 1e-5, 
                                  tolerance_change = 1e-5, 
                                  history_size = 100, 
                                  line_search_fn = 'strong_wolfe')


        train_model(max_iter,reps)


        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        alpha_full.append(alpha_val)    

        print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "alpha": alpha_full, "label": label}
    savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 4.3551474 Test MSE 386.1081110694545 Test RE 1.0016524210016506
1 Train Loss 3.871519 Test MSE 386.36081500226317 Test RE 1.0019801531782186
2 Train Loss 2.398406 Test MSE 384.22863603624273 Test RE 0.9992115537472298
3 Train Loss 2.386877 Test MSE 384.00347826307006 Test RE 0.998918742

96 Train Loss 2.3810084 Test MSE 383.80207397426915 Test RE 0.9986567485309908
97 Train Loss 2.3810084 Test MSE 383.80207397426915 Test RE 0.9986567485309908
98 Train Loss 2.3810084 Test MSE 383.80207397426915 Test RE 0.9986567485309908
99 Train Loss 2.3810084 Test MSE 383.80207397426915 Test RE 0.9986567485309908
Training time: 10.90
Training time: 10.90
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): 

92 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
93 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
94 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
95 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
96 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
97 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
98 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
99 Train Loss 2.3809218 Test MSE 383.74602020543455 Test RE 0.9985838196408576
Training time: 11.18
Training time: 11.18
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias

90 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
91 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
92 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
93 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
94 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
95 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
96 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
97 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
98 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
99 Train Loss 2.380847 Test MSE 383.7214881720682 Test RE 0.9985519005044761
Training time: 10.55
Training time: 10.55
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out

87 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
88 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
89 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
90 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
91 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
92 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
93 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
94 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
95 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
96 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
97 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
98 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381
99 Train Loss 2.3806276 Test MSE 383.658113443303 Test RE 0.9984694376102381

...

44 Train Loss 9442856000000.0 Test MSE 424.0981688570105 Test RE 1.049773924790898
45 Train Loss 9441011000000.0 Test MSE 424.0978360131966 Test RE 1.0497735128452084
46 Train Loss 9440988000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
47 Train Loss 9440988000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
48 Train Loss 9440987000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
49 Train Loss 9440987000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
50 Train Loss 9440988000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
51 Train Loss 9440988000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
52 Train Loss 9440987000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
53 Train Loss 9440987000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
54 Train Loss 9440988000000.0 Test MSE 424.09781395846915 Test RE 1.0497734855490666
55 Train Loss 9440987000000.0 Test MSE 424.09781395846915 Test RE 1.

60 Train Loss nan Test MSE nan Test RE nan
61 Train Loss nan Test MSE nan Test RE nan
62 Train Loss nan Test MSE nan Test RE nan
63 Train Loss nan Test MSE nan Test RE nan
64 Train Loss nan Test MSE nan Test RE nan
65 Train Loss nan Test MSE nan Test RE nan
66 Train Loss nan Test MSE nan Test RE nan
67 Train Loss nan Test MSE nan Test RE nan
68 Train Loss nan Test MSE nan Test RE nan
69 Train Loss nan Test MSE nan Test RE nan
70 Train Loss nan Test MSE nan Test RE nan
71 Train Loss nan Test MSE nan Test RE nan
72 Train Loss nan Test MSE nan Test RE nan
73 Train Loss nan Test MSE nan Test RE nan
74 Train Loss nan Test MSE nan Test RE nan
75 Train Loss nan Test MSE nan Test RE nan
76 Train Loss nan Test MSE nan Test RE nan
77 Train Loss nan Test MSE nan Test RE nan
78 Train Loss nan Test MSE nan Test RE nan
79 Train Loss nan Test MSE nan Test RE nan
80 Train Loss nan Test MSE nan Test RE nan
81 Train Loss nan Test MSE nan Test RE nan
82 Train Loss nan Test MSE nan Test RE nan
83 Train Lo

68 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
69 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
70 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
71 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
72 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
73 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
74 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
75 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
76 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
77 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
78 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE 0.9702447789336919
79 Train Loss 2683190000000.0 Test MSE 362.27424809521244 Test RE

57 Train Loss 2754638800000.0 Test MSE 406.45275313019977 Test RE 1.027702983543312
58 Train Loss 2751436700000.0 Test MSE 406.45511306904154 Test RE 1.0277059670544946
59 Train Loss 2748620500000.0 Test MSE 406.45546166314193 Test RE 1.0277064077577218
60 Train Loss 2731204700000.0 Test MSE 406.45359396223796 Test RE 1.0277040465514242
61 Train Loss 2718475500000.0 Test MSE 406.4497654128143 Test RE 1.027699206361721
62 Train Loss 2713109400000.0 Test MSE 406.444982274042 Test RE 1.0276931593138439
63 Train Loss 2706718600000.0 Test MSE 406.43654057192515 Test RE 1.0276824868674563
64 Train Loss 2702388800000.0 Test MSE 406.4345646425197 Test RE 1.0276799887769668
65 Train Loss 2701638500000.0 Test MSE 406.4351528152032 Test RE 1.0276807323813877
66 Train Loss 2700994200000.0 Test MSE 406.4352978584345 Test RE 1.0276809157539624
67 Train Loss 2700052000000.0 Test MSE 406.43641381750535 Test RE 1.027682326616969
68 Train Loss 2699386200000.0 Test MSE 406.4359833381682 Test RE 1.0276817

47 Train Loss 14137740000000.0 Test MSE 409.45459261020244 Test RE 1.03149103058008
48 Train Loss 13231063000000.0 Test MSE 409.4244983927607 Test RE 1.031453123463727
49 Train Loss 12482491000000.0 Test MSE 409.40600985570063 Test RE 1.0314298343412307
50 Train Loss 12085870000000.0 Test MSE 409.38928052664096 Test RE 1.0314087607554006
51 Train Loss 11878603000000.0 Test MSE 409.37273660288866 Test RE 1.0313879202973082
52 Train Loss 11758936000000.0 Test MSE 409.3504130380229 Test RE 1.0313597985307918
53 Train Loss 11671290000000.0 Test MSE 409.3313809510041 Test RE 1.0313358225471758
54 Train Loss 11513059000000.0 Test MSE 409.29826643500587 Test RE 1.0312941046641446
55 Train Loss 11403231000000.0 Test MSE 409.2781346860312 Test RE 1.0312687417307533
56 Train Loss 11242936000000.0 Test MSE 409.2648689442535 Test RE 1.0312520285783409
57 Train Loss 11022144000000.0 Test MSE 409.2550735115659 Test RE 1.0312396874014647
58 Train Loss 10795526000000.0 Test MSE 409.24895041673614 Test

In [12]:
import scipy.io as sio

In [17]:
re = np.zeros((25,1))
for tune_reps in range(25):
    label = "1D_SODE_Stan_tune"+str(tune_reps)+".mat" #WRONGLY SAVED AS STAN - DOESN'T MATTER
    data = sio.loadmat(label)
    re_array = np.array(data["test_re_loss"])
    re[tune_reps] =np.mean(re_array[:,-1]) 
print(re)

[[0.99859314]
 [1.01290119]
 [       nan]
 [       nan]
 [       nan]
 [0.99859278]
 [1.00334134]
 [       nan]
 [1.01769003]
 [       nan]
 [0.99849337]
 [0.98873285]
 [       nan]
 [1.01669173]
 [1.00630699]
 [       nan]
 [0.98672664]
 [       nan]
 [1.01799555]
 [1.01207527]
 [       nan]
 [       nan]
 [       nan]
 [0.99879528]
 [       nan]]


In [22]:
mini = np.nanmin(re)
for tune_reps in range(25):
    if(re[tune_reps] == mini):
        print(tune_reps)
        

16


In [24]:
re[16]

array([0.98672664])

In [25]:
lrn_tune[16]

array([0.5, 3. ])